In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [45]:
titanic_df = pd.read_csv('titanic/train.csv')
titanic_test_df = pd.read_csv('titanic/test.csv')

In [46]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
titanic_test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [48]:
useless_cols = ['PassengerId', 'Name', 'Ticket']
titanic_df = titanic_df.drop(columns=useless_cols, axis=1)

In [49]:
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [50]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 62.7+ KB


In [51]:
titanic_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [52]:
len(titanic_df[~titanic_df.Age.isnull()])

714

In [53]:
titanic_df.Parch.value_counts()

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

In [54]:
titanic_df.SibSp.value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

In [55]:
titanic_df.Pclass.value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [56]:
titanic_df[(titanic_df.Age.isnull()) & (titanic_df.Parch>1)]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
140,0,3,female,NaN,0,2,15.2458,NaN,C
159,0,3,male,NaN,8,2,69.5500,NaN,S
180,0,3,female,NaN,8,2,69.5500,NaN,S
201,0,3,male,NaN,8,2,69.5500,NaN,S
324,0,3,male,NaN,8,2,69.5500,NaN,S
533,1,3,female,NaN,0,2,22.3583,NaN,C
593,0,3,female,NaN,0,2,7.7500,NaN,Q
783,0,3,male,NaN,1,2,23.4500,NaN,S
792,0,3,female,NaN,8,2,69.5500,NaN,S
846,0,3,male,NaN,8,2,69.5500,NaN,S


In [57]:
default_embarked = titanic_df.Embarked.mode()[0]
default_age = titanic_df.Age.mode()[0]
default_cabin = titanic_df.Cabin.mode()[0]
titanic_df['Age_filled'] = titanic_df.groupby(['Sex', 'Pclass','Parch'])['Age']\
.transform(lambda x: x.fillna(x.median()))

In [58]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Sex           891 non-null object
Age           714 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Cabin         204 non-null object
Embarked      889 non-null object
Age_filled    891 non-null float64
dtypes: float64(3), int64(4), object(3)
memory usage: 69.7+ KB


In [59]:
#vals = {"C":0, "Q":1, "S":2, "NULL":99}
#titanic_df['Embarked_filled'] = titanic_df['Embarked'].map(lambda x: x if pd.isnull(x) else vals[x])
#titanic_df['Embarked_filled'] = titanic_df.groupby(['Survived', 'Sex', 'Pclass'])['Embarked_filled']\
#.transform(lambda x: x.fillna(x.median()))

In [62]:
encoder_embarked = LabelEncoder()
encoder_embarked.fit(titanic_df['Embarked'].fillna('NULL'))
titanic_df['Embarked_filled'] = titanic_df['Embarked'].map(lambda x: x if pd.isnull(x) else encoder_embarked.transform([x])[0])
titanic_df['Embarked_filled'] = titanic_df.groupby(['Sex', 'Pclass','Parch'])['Embarked_filled']\
.transform(lambda x: x.fillna(x.median()))
titanic_df['Embarked_filled'] = encoder_embarked.inverse_transform(titanic_df['Embarked_filled'].astype(int))

In [ ]:
titanic_df.Embarked_filled.value_counts()

In [ ]:
encoder_cabin = LabelEncoder()
encoder_cabin.fit(titanic_df['Cabin'].fillna('NULL'))
titanic_df['Cabin_filled'] = titanic_df['Cabin'].map(lambda x: x if pd.isnull(x) else encoder_cabin.transform([x])[0])
titanic_df['Cabin_filled'] = titanic_df.groupby(['Sex', 'Pclass'])['Cabin_filled']\
.transform(lambda x: x.fillna(x.median()))
titanic_df['Cabin_filled'] = encoder_cabin.inverse_transform(titanic_df['Cabin_filled'].astype(int))

In [ ]:
titanic_df.info()

In [ ]:
titanic_cleaned_df = titanic_df.drop(columns=['Age','Cabin','Embarked'], axis=1)

In [ ]:
titanic_cleaned_df.info()

In [ ]:
titanic_target = titanic_cleaned_df[['Survived']]
titanic_data = titanic_cleaned_df.drop(columns=['Survived'],axis=1)

In [ ]:
titanic_data.head()